Começamos removendo o arquivo `Iris.csv`, se ele já existe.


In [ ]:
!rm Iris.csv

# Para rodar o notebook, primeiramente baixe o [dataset](https://sourceforge.net/projects/irisdss/postdownload), que irá conter o arquivo ***iris.csv***.






Aqui criamos uma interface para upload do arquivo `Iris.csv` (assumimos que o arquivo vai ser enviado exatamente com esse nome)

In [ ]:
from google.colab import files
import io

# A variável upload é um dicionário com todos
# arquivos que foram enviados
uploaded = files.upload()

# A variável f é o arquivo de nome Iris.csv
# buscado no dicionário acima.
f = io.BytesIO(uploaded['Iris.csv'])

Saving Iris.csv to Iris.csv


Nas linhas abaixo zeramos o cursor do arquivo e lemos todas as linhas do arquivo em `lines`.

In [ ]:
# Zeramos o cursor para garantir que a leitura
# do arquivo inicie do início
f.seek(0)

# A variável lines é uma lista que contém as strings
# que representam cada linha do arquivo lido.
lines = f.readlines()

Abaixo criamos as matrizes dos dados de entrada e saída correspondentes, vazias mas já com as dimensões corretas.

In [ ]:
import numpy as np

# Aqui criamos as matrizes de entradas e saídas correspondentes
# ainda vazias
X = np.zeros((len(lines)-1,4)) # 4 entradas
Y = np.zeros((len(lines)-1,3)) # 3 saídas (one-hot encoding)

# Teremos 3 categorias. O vetor abaixo lista as strings
# que representam as categorias possíveis
cat = np.array(['Iris-setosa','Iris-versicolor','Iris-virginica'])

No laço abaixo populamos as matrizes `X` e `Y`.

In [ ]:
# Para cada linha do arquivo, exceto
# a primeira linha que é o cabeçalho
for i, line in enumerate(lines[1:]):

  # Aqui decodificamos a linha para transformar
  # de binário para caracteres ascii, e descartamos
  # o último caractere que representa uma nova linha
  s = line.decode()[:-1]

  # Aqui separamos os dados por vírgulas,
  # descartando o primeiro valor que é o id
  # pois usaremos i do laço como id.
  _,sl,sw,pl,pw,sp = s.split(',')

  # Transformamos as strings que representam
  # as dimensões de sépala e pétala para ponto
  # flutuante.
  sl = float(sl)
  sw = float(sw)
  pl = float(pl)
  pw = float(pw)
  
  # Aqui populamos as matrizes X e Y com os dados
  # coletados.
  X[i:] = np.array([sl,sw,pl,pw])
  Y[i:] = (cat == sp).astype('float') # Atenção para essa linha!

  # A última linha acima merece uma explicação mais longa:
  # Nessa linha fazemos um teste booleano, comparando cada
  # elemento do vetor cat com a string daquela linha do arquivo.
  # Para os elementos onde a comparação der verdadeiro, teremos
  # um booleano True, e para os elementos onde a comparação
  # der falso, teremos um booleano False. O resultado da comparação
  # do vetor cat com a string sp, na expressão cat == sp resulta
  # numa array booleana com mesmo formato que cat, mas com
  # valores booleanos True ou False em cada posição. O método
  # .astype('float') transforma o resultado em um array de float,
  # com 1.0 representando True e 0.0 representando False.
  # Então transformamos uma string como 'Iris-setosa' no vetor
  # [1.0, 0.0, 0.0] e uma string como 'Iris-versicolor' no vetor
  # [0.0, 1.0, 0.0] e assim por diante.

Nas linhas abaixo embaralhamos as amostras para não causar nenhum tipo de tendência no treinamento.

In [ ]:
import random

# Aqui criamos uma lista de índices
# embaralhados
indexes = list(range(150))
random.shuffle(indexes)

# Essa variável T indica quantas amostras
# serão usadas para treinamento. As demais
# serão usadas para validação
T = 140

# Aqui preparamos as matrizes dos pares
# de dados de treinamento e validação.
Xt = np.zeros((T,4))
Yt = np.zeros((T,3))
Xv = np.zeros((150-T,4))
Yv = np.zeros((150-T,3))

# Aqui preenchemos as matrizes com os
# respectivos valores
for i in range(0,T):
  Xt[i,:] = X[indexes[i],:]
  Yt[i,:] = Y[indexes[i],:]
for i in range(0,150-T):
  Xv[i,:] = X[indexes[T+i],:]
  Yv[i,:] = Y[indexes[T+i],:]

In [ ]:
# Essa será a função de ativação
# utilizada
def sigmoid(x):
  return 1.0 / (1.0 + np.exp(-x))

# O código abaixo implementa uma rede
# neural ariticial estilo perceptron
# com 4 entradas, 8 neurônios escondidos
# e 3 saídas
class Perceptron:
  def __init__(self):

    # Pesos e biases da entrada para a camada
    # escondida
    self.Wh = np.random.random((8,4))*2.0 - 1.0
    self.bh = np.random.random((8,1))*2.0 - 1.0

    # Pesos e bieases da camada escondida para
    # a saída
    self.Wo = np.random.random((3,8))*2.0 - 1.0
    self.bo = np.random.random((3,1))*2.0 - 1.0

    # Esse será o passo de aprendizagem
    self.eta = 0.1

  def forward(self,x):

    # Essa função faz o cálculo da saída
    # da rede neural no sentido direto.

    # Essa linha garante que x tenha tamanho
    # de 4 linhas e 1 coluna
    x = np.reshape(x,(4,1))

    # Calcula a soma ponderada para a camada escondida
    # somado ao bias
    self.sh = np.dot(self.Wh,x) + self.bh

    # Função de ativação é aplicada à camada escondida
    self.zh = sigmoid(self.sh)

    # Calcula a soma ponderada para a camada de saída
    # somado ao bias
    self.so = np.dot(self.Wo,self.zh) + self.bo

    # Função de ativação é aplciada à camada de saída
    self.zo = sigmoid(self.so)
    return self.zo # saída da rede ou saida da função de ativação da ultima cada IMPORTANTE APROVEITAR

  def train(self, X, Y):
    width , length = np.shape(Y) # formato do vetor de saida
    result_percep = np.zeros((width,length)) # Criando um vetor de zeros para armazenar os resultados de saida dos neurônios 
    error_final = 0
    n_amostras = width
    for i in range(n_amostras):
      x  = X[i,:] # Pegando os 4 valores de entrada
      x = np.reshape(x,(4,1))
      y = self.forward(x) # Aplicando a rede neural nos valores de entrada
      y_ = Y[i,:]
      y_ = np.reshape(y_,(3,1)) # Pegando os valores de saída dos 3 ultimos neurônios
      diff = y - y_
      error = np.sum((diff)**2) # Erro quadrático entre a saída obtida e esperada
      error_final = error_final + error
      # Calculando os deltas das camadas
      deltao =(diff)*(y*(1.0 - y))# Delta da camada escondida = diferença entra o calculado e estimada * derivada dos valores somados na camada
      deltah= np.dot(self.Wo.T, deltao)*(self.zh*(1.0 - self.zh)) # Delta da camada escondida = (pesos^t .* delta da camada de saida)*derivada dos valores da camada
      #Calculando os novos pesos
      self.Wo = self.Wo - self.eta*(deltao*self.zh.T)
      self.Wh = self.Wh - self.eta*(deltah*x.T)
      # Calculando os novos bias
      self.bo = self.bo - self.eta*deltao
      self.bh = self.bh - self.eta*deltah
      #l,w = np.shape(self.zh)
      #print(l)
      #print(w)
    error_final = error_final/n_amostras #Erro
    return error_final


In [ ]:
p = Perceptron()
p.train(Xt,Yt)

0.49622255168943213

Abaixo executamos o código, treinando a rede neural com os dados de treinamento separados anteriormente.

In [ ]:
# Criamos p que é nossa rede neural
p = Perceptron()

# Treinaremos 10 mil passos
for i in range(10000):
  
  # Aqui um passo de treinamento
  Err = p.train(Xt,Yt)

  # A cada mil passos mostramos o erro
  # total para verificar se está mesmo
  # diminuindo.
  if not (i % 1000) or i == 0:
    print('Err = ',Err)

Err =  0.6211764705245656
Err =  0.023759374131492345
Err =  0.015329792232192283
Err =  0.014665852642940333
Err =  0.01447146890572243
Err =  0.014373875380395067
Err =  0.014312504404928322
Err =  0.014267965073287434
Err =  0.014231507941509043
Err =  0.014197833239643753


Aqui vamos avaliar a performance dessa rede neural treinada, vendo como ela se sai nos dados separados para validação (dados para os quais ela nunca foi treinada).

In [ ]:
# Essa linha abaixo serve para suprimir a notação
# científica padrão do numpy para facilitar nosso
# olho humano comparar os resultados
np.set_printoptions(formatter={'float':lambda x: '%+01.2f ' % x})

# Para cada amostra de treinamento
# nesse laço
count = 0.0
n = Yv.shape[0] # Número de amostras
for i in range(150-T):
  y_ones = np.zeros(Yv.shape[1])
  # Separamos a entrada
  xv = Xv[i,:]

  # Calculamos a saída da rede
  y = p.forward(xv)

  # Separamos a saída esperada
  yv = Yv[i,:]
  idx = np.argmax(y)
  y_ones[idx] = 1
  idx2 = np.argmax(yv)
  if idx == idx2:
    count = count+1.0;
  # Mostramos ambos lado a lado

  
  if i==0:
    print('  Valores Obtidos','           Esperados','          Valores Obtidos(One hot encoding)')
  print(y.T, yv,y_ones)
count = count/n
# Porcentagem de acerto
print('\n ',count*100,'% de acerto')

  Valores Obtidos            Esperados           Valores Obtidos(One hot encoding)
[[+0.00  +0.02  +0.99 ]] [+0.00  +0.00  +1.00 ] [+0.00  +0.00  +1.00 ]
[[+1.00  +0.01  +0.00 ]] [+1.00  +0.00  +0.00 ] [+1.00  +0.00  +0.00 ]
[[+0.00  +1.00  +0.00 ]] [+0.00  +1.00  +0.00 ] [+0.00  +1.00  +0.00 ]
[[+1.00  +0.00  +0.00 ]] [+1.00  +0.00  +0.00 ] [+1.00  +0.00  +0.00 ]
[[+1.00  +0.01  +0.00 ]] [+1.00  +0.00  +0.00 ] [+1.00  +0.00  +0.00 ]
[[+0.00  +0.01  +0.99 ]] [+0.00  +0.00  +1.00 ] [+0.00  +0.00  +1.00 ]
[[+1.00  +0.00  +0.00 ]] [+1.00  +0.00  +0.00 ] [+1.00  +0.00  +0.00 ]
[[+0.00  +1.00  +0.00 ]] [+0.00  +1.00  +0.00 ] [+0.00  +1.00  +0.00 ]
[[+1.00  +0.00  +0.00 ]] [+1.00  +0.00  +0.00 ] [+1.00  +0.00  +0.00 ]
[[+0.00  +0.01  +0.99 ]] [+0.00  +0.00  +1.00 ] [+0.00  +0.00  +1.00 ]

 %f 100.0 % de acerto
